In [2]:
import csv
import pandas as pd
import os

Get average star of a shop in each month

In [6]:
'''
Get star of a shop in each month
:param business_id: The shop's id
:returns: Star of the shop every month as a list, the start month and the end month
'''
# DcBLYSvOuWcNReolRVr12A Drago's
business_id = 'DcBLYSvOuWcNReolRVr12A'
df = pd.read_csv("./new_orlean_restaurant_reviews.csv")
target_bus_df = df.loc[df['business_id'] == business_id]
print("number of reviews: ",len(target_bus_df.index))
target_bus_df = target_bus_df.sort_values(by=['date'])

start_year = int(target_bus_df.iloc[0].at['date'][0:4])
end_year = int(target_bus_df.iloc[-1].at['date'][0:4])
start_month = int(target_bus_df.iloc[0].at['date'][5:7])
end_month = int(target_bus_df.iloc[-1].at['date'][5:7])
# star - month dictionary re-scaled to 0 - (month_count-1)
star_month_dict = {}
for i in range(0,len(target_bus_df.index)):
    real_year = int(target_bus_df.iloc[i].at['date'][0:4])
    real_month = int(target_bus_df.iloc[i].at['date'][5:7])
    key_month = (real_year - start_year) * 12 + (real_month - start_month)
    if key_month in star_month_dict:
        star_month_dict[key_month].append(target_bus_df.iloc[i].at['stars'])
    else:
        star_month_dict[key_month] = [target_bus_df.iloc[i].at['stars']]
# print(star_month_dict)
# ensure every month has a average star
month_count = (end_year - start_year) * 12 + (end_month - start_month) + 1
per_month_stars = []
for key_month in range(month_count):
    if key_month in star_month_dict:
        per_month_stars.append(sum(star_month_dict[key_month])/len(star_month_dict[key_month]))
    else:
        per_month_stars.append(per_month_stars[key_month-1])

# print (per_month_stars, start_year, start_month, end_year, end_month)
print (start_year, start_month, end_year, end_month, month_count)
target_bus_df


number of reviews:  3217
2008 3 2022 1 167


,funny,useful,review_id,text,business_id,stars,date,user_id,cool
68797,0,3,nzmIa6HDRK43HHq2zoDu2Q,Martinis and lobster... now that's the good li...,DcBLYSvOuWcNReolRVr12A,4.0,2008-03-13 03:03:33,o-t-i7nbT5N_cmkCXs5oDQ,3
77460,0,0,HJbIab9q-e42ATXVPdhB8A,"We ordered the parmesan / butter oysters, alon...",DcBLYSvOuWcNReolRVr12A,4.0,2008-04-13 22:18:34,b9Azt2DNyTP9QJZdRFs8jw,1
60691,1,2,Wu7ihrc1bUerjr2vTr5nIw,Drago's is a Croatian family-owned restaurant ...,DcBLYSvOuWcNReolRVr12A,4.0,2008-06-08 14:13:02,MJqwVw8SyIuSdfqrg07QzA,2
58400,0,0,WdweC3kAy1HX9_KeE8W0Tg,ahhhhhh....c'mon gustav....leave nola alone......,DcBLYSvOuWcNReolRVr12A,4.0,2008-09-01 01:50:41,SxBi6fjWUCyIeDZd8jSvmQ,1
69885,0,1,48G80J0JMp6nrOVaLn3eKA,"Dear Home,\n\nI had BIG, MEATY oysters at this...",DcBLYSvOuWcNReolRVr12A,5.0,2008-09-24 18:21:41,OF1Dwu3fWeyNzX2ehlyM3A,0
...,...,...,...,...,...,...,...,...,...
89297,0,0,rKDuMoQ7ohc6dW0fTkJkDg,I was told about Drago's from a friend who rec...,DcBLYSvOuWcNReolRVr12A,4.0,2022-01-05 03:27:35,v24RjY1EMIqUXXoFbeYZXA,0
91071,0,0,-e7U_NiNLBIvURoG4xEcUA,We had reservations via Open Table and it took...,DcBLYSvOuWcNReolRVr12A,5.0,2022-01-07 15:33:36,1nMmVZocQ0j_8O4MAtTgOg,0
88949,0,0,z-Qp8w0JYQnQ4badseCsdg,We came here for my moms 70th bday. First our ...,DcBLYSvOuWcNReolRVr12A,3.0,2022-01-10 02:27:17,kV3mDYRNoB3KO8TBo7vEqw,0
90821,0,0,qxtn09m-eqymJh6KilEPtw,I was hesitant because I am not one for oyster...,DcBLYSvOuWcNReolRVr12A,4.0,2022-01-15 21:30:52,VH8f-I1_ka7NWJjZambcPQ,0


### Topic modelling of the shop's review in each month

In [13]:
import numpy as np
import pandas as  pd
from pprint import pprint# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# spaCy for preprocessing
import spacy
# import pyLDAvis.gensim
import gensim
# Plotting tools
import pyLDAvis
import matplotlib.pyplot as plt
%matplotlib inline

### Prepare stopwords  
We may visualize the cleaned corpus using wordcloud  


( run this only when the first time you use it )

In [ ]:
import nltk
nltk.download('stopwords') 

normal case just run below here

In [16]:
# NLTK Stop words
import nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Cpdo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


load data for training

In [26]:
# DcBLYSvOuWcNReolRVr12A Drago's
business_id = 'DcBLYSvOuWcNReolRVr12A'
df = pd.read_csv("./new_orlean_restaurant_reviews.csv")
target_bus_df = df.loc[df['business_id'] == business_id]
print("number of reviews: ",len(target_bus_df.index))
target_bus_df = target_bus_df.sort_values(by=['date'])

start_year = int(target_bus_df.iloc[0].at['date'][0:4])
end_year = int(target_bus_df.iloc[-1].at['date'][0:4])
start_month = int(target_bus_df.iloc[0].at['date'][5:7])
end_month = int(target_bus_df.iloc[-1].at['date'][5:7])
# text - month dictionary re-scaled to 0 - (month_count-1)
text_month_dict = {}
for i in range(0,len(target_bus_df.index)):
    real_year = int(target_bus_df.iloc[i].at['date'][0:4])
    real_month = int(target_bus_df.iloc[i].at['date'][5:7])
    key_month = (real_year - start_year) * 12 + (real_month - start_month)
    if key_month in text_month_dict:
        text_month_dict[key_month].append(target_bus_df.iloc[i].at['text'])
    else:
        text_month_dict[key_month] = [target_bus_df.iloc[i].at['text']]
        
# ensure every month has a average star
month_count = (end_year - start_year) * 12 + (end_month - start_month) + 1
per_month_texts = []
for key_month in range(month_count):
    if key_month in text_month_dict:
        per_month_texts.append(text_month_dict[key_month])
    else:
        per_month_texts.append(per_month_texts[key_month-1]) 
# print (per_month_stars, start_year, start_month, end_year, end_month)
print (start_year, start_month, end_year, end_month, month_count)


number of reviews:  3217
2008 3 2022 1 167


let's just choose a month with >=8 reviews as a test set

In [38]:
test_text = ""
for key_month in range(month_count):
    if len(per_month_texts[key_month]) >= 8:
        test_text = per_month_texts[key_month]
print("the test text is the month:",key_month,"th")
# test_text

the test text is the month: 166 th


Do some additional cleaning

In [40]:
# Remove ()
test_text = [sent.replace("("," ") for sent in test_text]
test_text = [sent.replace(")"," ")  for sent in test_text]    
# Remove new line characters 
test_text = [sent.replace("\n"," ") for sent in test_text]  
# Remove distracting single quotes 
test_text = [sent.replace("\'"," ") for sent in test_text]  
test_text

['Delicious food and excellent service! Ate from the small plates menu and all hit the mark. The charbroiled oysters melt in your mouth! Followed by the crawfish mac and cheese and gator tacos!',
 'We vacationed in NOLA for a week.. Came here for dinner. Had reservations and waited 15 minutes which wasn t horrible given dinner rush hours. After having chargrilled oysters at other nearby restaurants I feel they are all the same as other places here, all good and tasty. But the star were the fried shrimp and fish platter. The shrimp was so good! Lightly breaded, huge and Juicy and lots of flavor. The raw oysters were really fresh and sweet. Our waiter was very attentive and kind. Would definitely come back for the fried shrimp!',
 'Daaaaang, my mom hyped this place up for a reason! She has been to NOLA many times, and this was my 1st visit. She made sure I tried the best oysters in the city, and I was honestly not feeling it. I m not big into seafood, BUT.... these blew me away! The char

remove punctuations and tokenize words 

In [42]:
def sent_to_words(sentences):
  for sentence in sentences:
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))            #deacc=True removes punctuations
test_text = list(sent_to_words(test_text))
test_text

[['delicious',
  'food',
  'and',
  'excellent',
  'service',
  'ate',
  'from',
  'the',
  'small',
  'plates',
  'menu',
  'and',
  'all',
  'hit',
  'the',
  'mark',
  'the',
  'charbroiled',
  'oysters',
  'melt',
  'in',
  'your',
  'mouth',
  'followed',
  'by',
  'the',
  'crawfish',
  'mac',
  'and',
  'cheese',
  'and',
  'gator',
  'tacos'],
 ['we',
  'vacationed',
  'in',
  'nola',
  'for',
  'week',
  'came',
  'here',
  'for',
  'dinner',
  'had',
  'reservations',
  'and',
  'waited',
  'minutes',
  'which',
  'wasn',
  'horrible',
  'given',
  'dinner',
  'rush',
  'hours',
  'after',
  'having',
  'chargrilled',
  'oysters',
  'at',
  'other',
  'nearby',
  'restaurants',
  'feel',
  'they',
  'are',
  'all',
  'the',
  'same',
  'as',
  'other',
  'places',
  'here',
  'all',
  'good',
  'and',
  'tasty',
  'but',
  'the',
  'star',
  'were',
  'the',
  'fried',
  'shrimp',
  'and',
  'fish',
  'platter',
  'the',
  'shrimp',
  'was',
  'so',
  'good',
  'lightly',
  '

Phrases: Creating Bigram and Trigram models

In [53]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(test_text, min_count=1, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[test_text], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
# See examples
print(bigram_mod[test_text[1]])
# print(trigram_mod[bigram_mod[test_text[2]]])

['we', 'vacationed', 'in', 'nola', 'for', 'week', 'came', 'here', 'for', 'dinner', 'had', 'reservations', 'and', 'waited', 'minutes', 'which', 'wasn', 'horrible', 'given', 'dinner', 'rush', 'hours', 'after', 'having', 'chargrilled', 'oysters', 'at', 'other', 'nearby', 'restaurants', 'feel', 'they', 'are', 'all', 'the', 'same', 'as', 'other', 'places', 'here', 'all', 'good', 'and', 'tasty', 'but', 'the', 'star', 'were', 'the', 'fried', 'shrimp', 'and', 'fish', 'platter', 'the', 'shrimp', 'was', 'so', 'good', 'lightly', 'breaded', 'huge', 'and', 'juicy', 'and', 'lots', 'of', 'flavor', 'the', 'raw', 'oysters', 'were', 'really', 'fresh', 'and', 'sweet', 'our', 'waiter', 'was', 'very', 'attentive', 'and', 'kind', 'would', 'definitely', 'come_back', 'for', 'the', 'fried', 'shrimp']


Remove Stopwords, make bigrams and lemmatize

In [54]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

call  
if find fell to download spacy en  
download at:  
https://github.com/explosion/spacy-models/releases/tag/en_core_web_sm-3.3.0  
and install it with:  
pip install en_core_web_sm-3.3.0.tar.gz  

In [62]:
# Remove Stop Words
data_words_nostops = remove_stopwords(test_text)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

data_lemmatized

[['food',
  'excellent',
  'service',
  'eat',
  'small',
  'plate',
  'hit',
  'mark',
  'charbroile',
  'oyster',
  'melt',
  'mouth',
  'follow',
  'cheese',
  'gator'],
 ['vacation',
  'week',
  'come',
  'dinner',
  'reservation',
  'wait',
  'minute',
  'horrible',
  'give',
  'dinner',
  'rush',
  'hour',
  'chargrille',
  'oyster',
  'nearby',
  'restaurant',
  'feel',
  'place',
  'good',
  'tasty',
  'star',
  'fry',
  'fish',
  'good',
  'lightly',
  'bread',
  'huge',
  'juicy',
  'lot',
  'flavor',
  'raw',
  'oyster',
  'really',
  'fresh',
  'sweet',
  'waiter',
  'attentive',
  'kind',
  'definitely',
  'come_back',
  'fry',
  'shrimp'],
 ['hype',
  'place',
  'reason',
  'many',
  'time',
  'visit',
  'make',
  'sure',
  'try',
  'good',
  'oyster',
  'city',
  'honestly',
  'feel',
  'big',
  'seafood',
  'blow',
  'charbroil',
  'butter',
  'bread',
  'clean',
  'friend',
  'get',
  'cheese',
  'let',
  'try',
  'also',
  'world',
  'wait',
  'go',
  'back',
  'somed

Create Dictionary and Corpus needed for Topic Modeling  
Make sure to check if dictionary [id2word] or corpus is clean otherwise you may not get good quality topics.

In [67]:
# Create Dictionary 
id2word = corpora.Dictionary(data_lemmatized)  
# Create Corpus 
texts = data_lemmatized  
# Term Document Frequency 
corpus = [id2word.doc2bow(text) for text in texts]  
# View 
print(corpus[:1])
# (8,2) above indicates, word_id 8 occurs twice in the document and so on.
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1)]]


[[('charbroile', 1),
  ('cheese', 1),
  ('eat', 1),
  ('excellent', 1),
  ('follow', 1),
  ('food', 1),
  ('gator', 1),
  ('hit', 1),
  ('mark', 1),
  ('melt', 1),
  ('mouth', 1),
  ('oyster', 1),
  ('plate', 1),
  ('service', 1),
  ('small', 1)]]

### Topic: arameters of LDA

Alpha and Beta are Hyperparameters — alpha represents document-topic density and Beta represents topic-word density, chunksize is the number of documents to be used in each training chunk, update_every determines how often the model parameters should be updated and passes is the total number of training passes.  
A measure for best number of topics really depends on kind of corpus you are using, the size of corpus, number of topics you expect to see.

In [68]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

Fast view of topics

In [69]:
# Print the keyword of topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.013*"oyster" + 0.013*"service" + 0.013*"food" + 0.013*"charbroile" + '
  '0.013*"plate" + 0.013*"excellent" + 0.013*"cheese" + 0.013*"eat" + '
  '0.013*"small" + 0.013*"mark"'),
 (1,
  '0.027*"oyster" + 0.027*"fry" + 0.018*"seafood" + 0.018*"try" + 0.014*"good" '
  '+ 0.014*"great" + 0.014*"order" + 0.014*"even" + 0.014*"well" + '
  '0.010*"come"'),
 (2,
  '0.033*"oyster" + 0.019*"bread" + 0.017*"good" + 0.017*"place" + 0.017*"try" '
  '+ 0.011*"make" + 0.011*"definitely" + 0.011*"great" + 0.011*"gumbo" + '
  '0.011*"also"')]


You can see the top keywords and weights associated with keywords contributing to topic.  
Topics are words with highest probability in topic and the numbers are the probabilities of words appearing in topic distribution.  


Topic evaluation:  
Lower the perplexity better the model.  
Higher the topic coherence, the topic is more human interpretable  

In [71]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -5.752391898402801

Coherence Score:  0.43376822932112163


A maybe useful visualization

In [74]:
# Visualize the topics
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

c:\Users\Cpdo\.conda\envs\ARTS\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.085587 -0.014780       1        1  64.159484
1      0.088771 -0.013245       2        1  32.529311
0     -0.003184  0.028024       3        1   3.311205, topic_info=           Term      Freq     Total Category  logprob  loglift
3     excellent  2.000000  2.000000  Default  30.0000  30.0000
5          food  3.000000  3.000000  Default  29.0000  29.0000
0    charbroile  3.000000  3.000000  Default  28.0000  28.0000
13      service  2.000000  2.000000  Default  27.0000  27.0000
1        cheese  1.000000  1.000000  Default  26.0000  26.0000
..          ...       ...       ...      ...      ...      ...
142       first  0.049666  1.911727   Topic3  -5.7149  -0.2426
16        bread  0.050071  5.906459   Topic3  -5.7068  -1.3625
74      seafood  0.049984  4.568125   Topic3  -5.7085  -1.1073
38        place  0.049964  5.136493   Topic3  -5.7089  -1.2250
52         also  0.049956  3.594281   Topic3  -5.7091  -0.8681

[176 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
203       2  0.874016         allergy
160       2  0.564634  alligator_bite
161       2  0.875141          almost
52        1  0.834659            also
205       2  0.873983             bar
...     ...       ...             ...
201       2  0.874863             way
136       1  0.508080            well
136       2  0.508080            well
202       2  0.875106            whip
218       2  0.873925            work

[127 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])